In [9]:
import random
import time

k         = 3
dimension = 3

path = "foo.txt"
splitChar = " "

data = sc.textFile(path).map(lambda line: [[float(i) for i in line.split(splitChar)],0])
centers = []
timeA = time.time()
centers.append(data.takeSample(False,1,random.randint(1,1000))[0][0])
while(len(centers)!=k):
    data = data.map(lambda x: [x[0],min([sum([(x[0][i]-c[i])**2 for i in range(dimension)]) for c in centers])])
    sumD = data.map(lambda x:x[-1]).reduce(lambda x,y:x+y)
    d = random.randint(1,sumD)
    
    datalist = data.collect()
    for i in datalist:
        d -= i[-1]
        if d <= 0:
            centers.append(i[0])
            break
timeB = time.time()
print centers

[[44.0, 52.0, 20.0], [46.0, 12.0, 169.0], [107.0, 168.0, 128.0]]


In [10]:
print timeB-timeA

12.2852101326


In [14]:
a = 14.4359109402 + 18.5155801773
b = 13.2317810059 + 27.8765740395
c = 12.2946197987 + 26.5864329338

d = 11.1266639233 + 23.6135261059
e = 12.9403269291 + 23.5073430538
f = 12.2852101326 + 34.5989868641

print [a,b,c,d,e,f],sum([a,b,c,d,e,f])/6

a = [262048141.0,189392728.172,189392728.172,189392728.172,189392728.172,189392728.172,189392728.172,189392728.172,189392728.172]
b = [305773157.0,189392700.01,189392700.01,189392700.01,189392700.01,189392700.01,189392700.01,189392700.01,189392700.01]
c = [355515639.0,189393098.442,189393098.442,189393098.442,189393098.442,189393098.442,189393098.442,189393098.442,189393098.442]
cSum = [0 for i in range(9)]
for i in range(9):
    cSum[i] = (a[i] + b[i] + c[i])/3
print cSum



[32.9514911175, 41.1083550454, 38.8810527325, 34.7401900292, 36.447669982899995, 46.8841969967] 38.5021593174
[307778979.0, 189392842.20799997, 189392842.20799997, 189392842.20799997, 189392842.20799997, 189392842.20799997, 189392842.20799997, 189392842.20799997, 189392842.20799997]


In [24]:
import random
import time
import copy

k = 3
dimension = 3
threshold = 1e-4

def updateCluster(u,centerT):
    costMin = 999999
    pos     = -1
    for i in range(3):
        cost = sum([(u[j]-centerT[i][j])**2 for j in range(3)])
        if cost <costMin:
            costMin = cost
            pos = i
    return [u,pos+1,costMin]

timeE = time.time()
data = sc.textFile("foo.txt").map(lambda line: [float(i) for i in line.split(" ")]).cache()
#centers = [[44.0, 52.0, 20.0], [46.0, 12.0, 169.0], [107.0, 168.0, 128.0]]
centers = data.takeSample(False,k,random.randint(1,1000))
costlist = []
costTotal = data.map(lambda x: updateCluster(x,centers)[-1]).reduce(lambda x,y:x+y)
print costTotal
costlist.append(costTotal)
iterCount = 0

while True:
    iterCount += 1
    centerN = copy.deepcopy(centers)
    for i in range(k):
        cluster = data.map(lambda x: updateCluster(x,centers)).filter(lambda x:x[1]==i+1).map(lambda x:x[0]).reduce(lambda x,y:[x[j]+y[j] for j in range(dimension)])
        clusterSize = data.map(lambda x: updateCluster(x,centers)).filter(lambda x:x[1]==i+1).count()
        centerN[i] = [cluster[h]/clusterSize for h in range(dimension)]
    costN = data.map(lambda x: updateCluster(x,centerN)).map(lambda x: x[-1]).reduce(lambda x,y: x+y)
 
    if(costN < (1-threshold)*costTotal):
        costTotal = costN
        costlist.append(costTotal)
        centers = copy.deepcopy(centerN)
    else:
        break

print iterCount, costlist

timeF = time.time()

print timeF-timeE

836363859.0
5 [836363859.0, 461227366.1158758, 289848428.8005171, 193290313.61561763, 189392700.00978732]
39.9129331112


In [25]:
a1 = [807585875.0, 652665999.0416747, 652418238.322978, 651952844.7492521, 650768854.5589807, 649718232.6404512, 649378041.6710244, 649306278.7431269]
t1 = 69.6942808628
a2 = [1492729219.0, 420004224.07864124, 199384240.95665947, 189392700.00978732,189392700.00978732, 189392700.00978732, 189392700.00978732, 189392700.00978732]
t2 = 33.4973571301
a3 = [1530223130.0, 689564111.853937, 512402253.6531409, 270442828.3794469, 190357373.4491771, 189392700.00978732, 189392700.00978732,189392700.00978732]
t3 = 47.8724839687
a4 = [1409721265.0, 583657299.0962943, 189392700.00978732, 189392700.00978732, 189392700.00978732, 189392700.00978732, 189392700.00978732, 189392700.00978732]
t4 = 25.7655501366
a5 = [940021524.0, 653635152.457326, 650661064.1422832, 649644770.5342073, 649372138.5372837,649372138.5372837,649372138.5372837,649372138.5372837]
t5 = 45.6077339649

costs = []
for i in range(len(a1)):
    costs.append((a1[i]+a2[i]+a3[i]+a4[i]+a5[i])/5)
print costs
t6 = 34.5989868641
t7 = 55.9014999866
t8 = 38.4159219265
t9 = 39.9129331112
print sum([t1,t2,t3,t4,t5,t6,t7,t8,t9])/9

[1236056202.6, 599905357.3055747, 440851699.4169698, 390165168.7364962, 373856753.3130032, 373453694.2414194, 373385656.04753405, 373371303.4619545]
43.4740831057


In [16]:
# no store
import random
import time

k         = 3
dimension = 3

path = "foo.txt"
splitChar = " "

data = sc.textFile(path).map(lambda line: [[float(i) for i in line.split(splitChar)],0])
centers = []
timeA = time.time()
centers.append(data.takeSample(False,1,random.randint(1,1000))[0][0])
while(len(centers)!=k):
    sumD = data.map(lambda x: [x[0],min([sum([(x[0][i]-c[i])**2 for i in range(dimension)]) for c in centers])]).map(lambda x:x[-1]).reduce(lambda x,y:x+y)
    d = random.randint(1,sumD)
    
    datalist = data.map(lambda x: [x[0],min([sum([(x[0][i]-c[i])**2 for i in range(dimension)]) for c in centers])]).collect()
    for i in datalist:
        d -= i[-1]
        if d <= 0:
            centers.append(i[0])
            break
timeB = time.time()
print centers

[[34.0, 57.0, 178.0], [16.0, 30.0, 28.0], [79.0, 52.0, 168.0]]


In [17]:
print timeB-timeA

14.3522219658
